In [ ]:
from Bio.motifs import meme
from Bio import motifs
file_meme_xml="/home/davide/Desktop/genomiChro/MEME/motivo8_oops/meme.xml"
# meme_record = meme.read(open(file_meme_xml)) # stessa cosa di sotto in un altro modo
meme_record = motifs.parse(open(file_meme_xml), 'meme')
motivo = meme_record[0]

In [ ]:
print(motivo.consensus)
# sequenza consenso del motivo

In [ ]:
print(motivo.degenerate_consensus)
# sequenza consenso degenerato del motivo

In [ ]:
print(motivo.counts)
#matrice di conteggio delle basi per ogni posizione del motivo

In [ ]:
print(motivo.pwm)
#matrice di probabilità delle basi per ogni posizione del motivo

In [ ]:
print(motivo.name)
#nome del motivo presente nel file meme.xml

In [ ]:
print(motivo.length)
#lunghezza del motivo

In [ ]:
print(motivo.alignment)
#allineamento del motivo con le sequenze con cui è stato formato

In [ ]:
print(motivo.relative_entropy)

In [ ]:

motivo.pseudocounts=1
print(motivo.pseudocounts)

In [ ]:
print(motivo.relative_entropy)

In [ ]:
print(motivo.pwm)

In [ ]:
from math import log
from Bio import motifs
import Bio
def sm(motivo:Bio.motifs, seq:str):
    """
    Calcola lo score SM di una sequenza rispetto ad un motivo
    :param motivo:  il motivo in formato Bio.motifs
    :param seq:     la sequenza su cui calcolare lo score
    :return:    lo score della sequenza rispetto al motivo
    """
    q=motivo.background #frequenze delle basi in tutte le sequenze intergeniche
    pwm=motivo.pwm #matrice di probabilità delle basi per ogni posizione del motivo
    def a():
        """
        Calcola il fattore di normalizzazione a
        :return:    il fattore di normalizzazione a
        """
        n=len(motivo.instances) #numero di sequenze con cui è stato costruito il motivo
        a=(n+1)/(n+4)*log(n+1)-log(n+4)-1/(n+4)*sum(log(q[b]) for b in "ACGT")-n/(n+4)*log(min(q.values()))
        return a
    a=a() #fattore di normalizzazione a
    
    def Info(i):
        """ Calcola l'entropia relativa per la posizione i del motivo"""
        somma = sum(pwm[b,i]*log(pwm[b,i]/q[b]) for b in "ACGT")
        return somma/a
    
    max=-float("inf")
    max_i=0
    for i in range(len(seq)-motivo.length+1):
        h=seq[i:i+motivo.length] #sottosequenza di lunghezza del motivo(l-mero della sequenza intergenica)
        score=sum(Info(i)*log(pwm[h[i],i]/q[h[i]]) for i in range(len(h))) #score della sottosequenza
        if score>max:
            max=score
            max_i=i
    return max,seq[max_i:max_i+motivo.length],max_i-len(seq)

# sequenza consenso del motivo
for record in SeqIO.parse("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/ortologhi/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta","fasta"):
    if "WP_250121220.1" in record.description:
        print(sm(motivo,record.seq))

In [ ]:
def Hamming(s1:str,s2:str):
    """
    Calcola la distanza di Hamming tra due sequenze
    :param s1:  la prima sequenza
    :param s2:  la seconda sequenza
    :return:    la distanza di Hamming tra le due sequenze
    """
    return sum(1 for i in range(len(s1)) if s1[i]!=s2[i])
from pathlib import Path
from Bio import SeqIO

def ortologhi(pid:str):
    """
    Restituisce la lista delle sequenze intergeniche upstream degli ortologhi del gene
    :param pid: l'identificatore del gene(Protein ID)
    :return:    la lista delle sequenze intergeniche upstream degli ortologhi del gene
    """
    seq_intergeniche=Path("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/ortologhi")
    seqs=[]
    for file in seq_intergeniche.iterdir():
        if "CCMEE_29" not in file.name:
            for record in SeqIO.parse(file,"fasta"):
                if pid in record.description:
                    seqs.append(record.seq)
    return seqs

def refined_score(motivo:Bio.motifs, pid:str, intergenica:str):
    """
    Calcola lo score SM di una sequenza rispetto ad un motivo
    :param motivo:  il motivo in formato Bio.motifs
    :param pid:     l'identificatore del gene(Protein ID)
    :return:    lo score della sequenza rispetto al motivo
    """
    intergeniche_ortologhi=ortologhi(pid)
    motivo_len=motivo.length
    s=sm(motivo,intergenica)[1] #sequenza  nell'intergenica di CCMEE29 con maggior score
    score_ortologhi=[(motivo_len-Hamming(s,seq))/motivo_len*sm(motivo,seq)[0] for seq in intergeniche_ortologhi]
    avg=sum(score_ortologhi)/len(score_ortologhi)
    return sm(motivo,intergenica)[0]+avg
    


In [ ]:
for record in SeqIO.parse("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/ortologhi/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta","fasta"):
    if "WP_250121220.1" in record.description:
        s=record.seq
        print(refined_score(motivo,"WP_250121220.1",s))